In [ ]:
TaskObject = "{\"TaskInstanceId\":53,\"TaskMasterId\":1,\"TaskStatus\":\"InProgress\",\"TaskType\":\"Execute Synapse Notebook\",\"Enabled\":1,\"ExecutionUid\":\"dc872650-b992-4cae-9ae2-c714c95563ee\",\"NumberOfRetries\":2,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ads-dev-kv-ads-tad5.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"Source\":{\"System\":{\"SystemId\":-4,\"SystemServer\":\"https://adsdevdlsadstad5adsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakeraw\"},\"Instance\":{\"SourceRelativePath\":\"/samples/sif/\",\"TargetRelativePath\":\"/samples/sif/StudentPersonal/\"},\"DataFileName\":\"StudentPersonal.json\",\"RelativePath\":\"/samples/sif/\",\"SchemaFileName\":\"\",\"Type\":\"Notebook-Optional\",\"WriteSchemaToPurview\":\"Disabled\"},\"Target\":{\"System\":{\"SystemId\":-4,\"SystemServer\":\"https://adsdevdlsadstad5adsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakeraw\"},\"Instance\":{\"SourceRelativePath\":\"/samples/sif/\",\"TargetRelativePath\":\"/samples/sif/StudentPersonal/\"},\"DataFileName\":\"StudentPersonal.parquet\",\"RelativePath\":\"/samples/sif/StudentPersonal/\",\"SchemaFileName\":\"\",\"Type\":\"Notebook-Optional\",\"WriteSchemaToPurview\":\"Disabled\"},\"TMOptionals\":{\"CustomDefinitions\":\"\",\"ExecuteNotebook\":\"SIFParameterizedJson\",\"Purview\":\"Disabled\",\"QualifiedIDAssociation\":\"TaskMasterId\",\"UseNotebookActivity\":\"Enabled\"}}"

In [ ]:
TaskObject

In [ ]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

session_id = random.randint(0,1000000)
#invalid source
##TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\",\"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-irud.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ark-stg-adf-ads-irud\",\"SystemType\":\"Datafactory\",\"ResourceGroup\":\"dlzdev04\",\"SubscriptionId\":\"ed1206e0-17c7-4bc2-ad4b-f8d4dab9284f\",\"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"}}"
#valid source
#TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\", \"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1, \"KeyVaultBaseUrl\":\"https://ads-dev-kv-ads-ic038069.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\", \"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ads-dev-kv-ads-ic038069\", \"SystemType\":\"Microsoft.Synapse/workspaces\",\"ResourceGroup\":\"sifgofast\",\"SubscriptionId\":\"cd486ba9-eef3-466d-b16c-7f1b2941ae9d\", \"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\", \"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://adsdevdlsadsic03adsl.blob.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"},\"Instance\":\"\",{\"TargetRelativePath\":\"synapse/sif\"}, \"DataFileName\":\"StudentPersonal.parquet\",\"SourceRelativePath\":\"synapse/sif\",\"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Parquet\"}, \"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://adsdevdlsadsic03adsl.blob.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ABS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"}, \"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"StudentPersonal.parquet\",\"SourceRelativePath\":\"synapse\/sif\", \"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Parquet\"}}"
TaskDict = {}
OutputDict = {}
TaskObjectJson = json.loads(TaskObject)



##we want to delete EngineJson as it causes issues when converting back to a json and it is not needed as its properties are within JsonProperties as children
try:
    del TaskObjectJson['ExecutionEngine']['EngineJson']
except:
    print("No EngineJson Found")

Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","").replace("blob","dfs") + "/"
Source = Source.replace('///', '/')
Source = Source.replace('//', '/')
print(Source)
Source = Source + TaskObjectJson['Source']["Instance"]['SourceRelativePath']
print(Source)
Source = Source  +"/" + TaskObjectJson['Source']['DataFileName'] 
print(Source)
Source = Source.replace('///', '/')
Source = Source.replace('//', '/')
print(Source)
Source = "abfss://" + Source 
print(Source)

In [ ]:
Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","").replace("blob","dfs") + "/"
Target = Target.replace('///', '/')
Target = Target.replace('//', '/')
print(Target)
Target = Target + TaskObjectJson['Target']["Instance"]['TargetRelativePath']
Target = Target  +"/" + TaskObjectJson['Target']['DataFileName'] 
print(Target)
Target = Target.replace('///', '/')
Target = Target.replace('//', '/')
print(Target)
Target = "abfss://" + Target 
print(Target)

In [ ]:
from pyspark.sql.functions import col, explode_outer, from_json, lit, concat
from pyspark.sql.types import StructType, ArrayType
print(Source)
input_df = spark.read.option("multiline","true").json(Source)
#display(input_df)


#output_df = execute_autoflatten_with_PK(input_df,None)
#display(output_df)



In [ ]:
from delta.tables import *

df = input_df

#SIF model uses RefId as Unique identifier in all the structures
mergeCondition = "oldData.RefId = newData.RefId"




print("SourceDT = " + TaskObjectJson['Source']['Type']  + ", TargetDT = " + TaskObjectJson['Target']['Type'] )
#df = spark.read.load(Source, format=SourceDT)

var_check = DeltaTable.isDeltaTable(spark, Target )
try:
    if (var_check):
        print("Performing Merge... on Existing table")

        olddt = DeltaTable.forPath(spark, Target) 

        olddt.alias("oldData").merge(
            df.alias("newData"),
            mergeCondition) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    else:
        print("Creating new Delta Table.")    
        df.write.format("Delta").save(Target)
except:
    print("Table does not exist. Creating new Delta Table.")    
    df.write.format("Delta").save(Target)


In [ ]:
print("Creating Spark Table")
df = spark.read.load(Target, format='delta')
targetDB = "sif"
targetTable = "raw_" + TaskObjectJson['Target']['DataFileName'].split(".")[0] 
#if the target datatype is parquet then we do not need to create a copy of the data - we can use the recently saved sink target
if (TaskObjectJson['Target']['Type'] == 'Parquet'):
    SnapshotTarget = Target
else:
    SnapshotTarget = Target + '/Snapshot/' + targetTable
    #we need to update the parquet file - this is not very efficient but there isnt a current better way as delta tables are not supported for persistent tables
    df.write.format("parquet").mode("overwrite").save(SnapshotTarget)


#we need to make the DB and table lowercase as synapse persistent tables dont identify them as different identities
targetDB = targetDB.lower()
targetTable = targetTable.lower()

#check if the specified DB / table exists - if so only do required actions.
dbList = spark.catalog.listDatabases()
dbExists = False
for db in dbList:
    if (db.name == targetDB):
        dbExists = True
        break
if (dbExists):
    print("DB Exists")
    tableExists = False
    spark.catalog.setCurrentDatabase(targetDB)
    tableList = spark.catalog.listTables()
    for table in tableList:
        if (table.name == targetTable):
            tableExists = True
            break
    if (tableExists):
        print("Table exists - nothing needed to be done")
        spark.catalog.refreshTable(targetTable)
    else:
        print("Table doesnt exist - creating")
        spark.catalog.createExternalTable(targetTable, path=SnapshotTarget, source='parquet')
else:
    print("DB Doesnt exist - creating DB and table")
    createDBString = "CREATE DATABASE " + targetDB 
    spark.sql(createDBString)
    spark.catalog.setCurrentDatabase(targetDB)
    spark.catalog.createExternalTable(targetTable, path=SnapshotTarget, source='parquet')